In [11]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from data_process.column_schema import (PROPERTIES_RENAME_DICT, TRANSACTION_RENAME_DICT, 
                                        NUMERICAL_COLS, CATEGORICAL_COLS)
from data_process.data_process_pipeline import DataProcessPipeline
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
df_all = pd.read_csv('/Users/shuyangdu/Desktop/ZillowChallenge/data/df_merged.csv')

In [3]:
data_pipeline = DataProcessPipeline(encode_mode='label')

In [4]:
df = data_pipeline.pre_process(df_all)

In [5]:
iterator = data_pipeline.k_fold(df, 52)

In [6]:
df_train, df_val = iterator.next()

In [7]:
df_train = data_pipeline.post_process(df_train, is_train=True)
df_val = data_pipeline.post_process(df_val, is_train=False)

In [9]:
X_train = df_train[data_pipeline.feature_cols].values
y_train = df_train[data_pipeline.label_col].values
X_val = df_val[data_pipeline.feature_cols].values
y_val = df_val[data_pipeline.label_col].values

# Construct Model

In [78]:
model = LGBMRegressor(objective='regression_l1', n_estimators=50, num_leaves=55, learning_rate=0.05,
                      feature_fraction=1.0, bagging_freq=6, bagging_fraction=0.7, lambda_l2=27)

In [77]:
model.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='l1')

[1]	valid_0's l1: 0.0671784
[2]	valid_0's l1: 0.0671055
[3]	valid_0's l1: 0.0670476
[4]	valid_0's l1: 0.0669909
[5]	valid_0's l1: 0.0669418
[6]	valid_0's l1: 0.0668945
[7]	valid_0's l1: 0.0668554
[8]	valid_0's l1: 0.0668182
[9]	valid_0's l1: 0.0667895
[10]	valid_0's l1: 0.0667585
[11]	valid_0's l1: 0.0667018
[12]	valid_0's l1: 0.0666773
[13]	valid_0's l1: 0.0666504
[14]	valid_0's l1: 0.0666217
[15]	valid_0's l1: 0.0666015
[16]	valid_0's l1: 0.0665803
[17]	valid_0's l1: 0.0665636
[18]	valid_0's l1: 0.06655
[19]	valid_0's l1: 0.0665436
[20]	valid_0's l1: 0.0665378
[21]	valid_0's l1: 0.0665186
[22]	valid_0's l1: 0.0665018
[23]	valid_0's l1: 0.0664879
[24]	valid_0's l1: 0.0664724
[25]	valid_0's l1: 0.0664626
[26]	valid_0's l1: 0.0664524
[27]	valid_0's l1: 0.0664408
[28]	valid_0's l1: 0.06643
[29]	valid_0's l1: 0.0664346
[30]	valid_0's l1: 0.0664239
[31]	valid_0's l1: 0.0664178
[32]	valid_0's l1: 0.066416
[33]	valid_0's l1: 0.066412
[34]	valid_0's l1: 0.0664072
[35]	valid_0's l1: 0.0663984


LGBMRegressor(bagging_fraction=0.8, bagging_freq=5, boosting_type='gbdt',
       colsample_bytree=1, feature_fraction=1.0, learning_rate=0.05,
       max_bin=255, max_depth=-1, min_child_samples=10, min_child_weight=5,
       min_split_gain=0, n_estimators=50, nthread=-1, num_leaves=31,
       objective='regression_l1', reg_alpha=0, reg_lambda=0, seed=0,
       silent=True, subsample=1, subsample_for_bin=50000, subsample_freq=1)

In [43]:
y_pred = model.predict(X_val)

In [44]:
mean_absolute_error(y_val, y_pred)

0.066439949328684508

In [45]:
model.fit(df_train[data_pipeline.feature_cols], df_train[data_pipeline.label_col],
          feature_name = data_pipeline.feature_cols,
          categorical_feature=CATEGORICAL_COLS)

LGBMRegressor(boosting_type='gbdt', colsample_bytree=1, learning_rate=0.05,
       max_bin=255, max_depth=-1, min_child_samples=10, min_child_weight=5,
       min_split_gain=0, n_estimators=30, nthread=-1, num_leaves=31,
       objective='regression_l1', reg_alpha=0, reg_lambda=0, seed=0,
       silent=True, subsample=1, subsample_for_bin=50000, subsample_freq=1)

In [46]:
y_pred = model.predict(X_val)

In [47]:
mean_absolute_error(y_val, y_pred)

0.06663904309064908